# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [4]:
# Before splitting the data into train and test, ensure it is all numeric to be used by the models
# Quick scan of the data shows that homeowner column is having text values (rent, own, mortgage). 
# 1 method: Fit the values using label encoder and then apply that to the column using transform

# Creating an instance of label encoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Fitting the label encoder
label_encoder.fit(df["homeowner"])

# List the classes identified by the label encoder
list(label_encoder.classes_)

df["homeowner_le"] = label_encoder.transform(df["homeowner"])
df.head()

# 2nd method: When it is more meaningful to assign specific values we can use a dictionary and 
# lambda as below. Keeping it in comments for futre refernce

## Checke the possible values - rent own and mortgage
## df["homeowner"].unique()
## Define a dictionary 
##ownership_num = {
##    "rent": 1,
##    "own": 2,
##    "mortgage": 3,
##}

## homeowner values encoded using the dictionary values
## df["homeowner_num"] = df["homeowner"].apply(lambda x: ownership_num[x])
## df.head()


,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status,homeowner_le
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk,1
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk,1
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk,2
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk,1
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk,0


# Split the Data into Training and Testing

In [5]:
# Create our features
# Assign all columns from df to our X except loan_status (which would be y)
# and homeowner becauses encoded version homeowner_le is already there
X = df.copy()
X.drop(["loan_status", "homeowner"], axis=1, inplace=True)
X.head()

# Create our target. This is what we are solving for
y = df["loan_status"]
y.head()


0    low_risk
1    low_risk
2    low_risk
3    low_risk
4    low_risk
Name: loan_status, dtype: object

In [6]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_le
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.606144
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.667811
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,1.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,2.000000


In [7]:
# Check the balance of our target values
##y['loan_status'].value_counts()
y.value_counts()


low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [8]:
# Create X_train, X_test, y_train, y_test (Splitting X and y into Train and Test sets)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X_train.shape

(58152, 8)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [11]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
## X_train_scaled[:5]

# Below can be used to check mean and std deviation of the data that they indeed equal 0 and 1
##round(X_train_scaled.mean(), 0)
##round(X_train_scaled.std(), 0)

# Set X_train and X_test to scaled values to run models on scaled X sets. 
# Can comment below 2 lines to run on unscaled X and compare
X_train = X_train_scaled
X_test = X_test_scaled

# Simple Logistic Regression

In [12]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [13]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9889115309798473

In [14]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  609,    10],
       [  113, 18652]], dtype=int64)

In [15]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.98      0.99      0.91      0.99      0.98       619
   low_risk       1.00      0.99      0.98      1.00      0.99      0.98     18765

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [16]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [17]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [18]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9934649587814939

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  615,     4],
       [  124, 18641]], dtype=int64)

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [21]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)
from collections import Counter

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [22]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [23]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9934116680807212

In [24]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  615,     4],
       [  126, 18639]], dtype=int64)

In [25]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.90      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [26]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

from collections import Counter

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 1881, 'low_risk': 1881})

In [27]:
# Train the Logistic Regression model using the resampled data

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [28]:
# Calculate the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9934116680807212

In [29]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  613,     6],
       [  113, 18652]], dtype=int64)

In [30]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.98       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.98     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.98     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [31]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 55687, 'low_risk': 55947})

In [32]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [33]:
# Calculate the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9921425487504449

In [34]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  615,     4],
       [  124, 18641]], dtype=int64)

In [35]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      0.99      0.99      0.91      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

Based on the Balanced Accuracy Scores for the models used above on our data:  

Answer: 
Logistic Regression Model fares the lowest in terms of balanced accuracy score. 
Within in the 2 models for oversampling- the NAIVE fares tiny bit better than the SMOTE.
Undersampling (ClusterCentroids) fares exactly the same as SMOTE OverSampling.
SMOTEENN (Under and Over Sampling) fares better than the Logistic Regression but worse than the others.
   
Note 1: All values are very close to 1 (indicating a good score) but when comparing them with each other, I have compared right down to the last decimal place in making the statements above. 

Note 2: Re-Running all the models with unscaled X train and X test data keeps the observations similar but further lowers the balanced acucuracy score with the Logistic Regression.

A. With scaled X_train and X_test

LogisticRegression 0.9889115309798473
OverSampling Naive 0.9934649587814939, SMOTE OverSampling 0.9934116680807212
ClusterCentroids UnderSampling 0.9934116680807212 
SMOTEENN Combination - Over and Under Sampling 0.9921425487504449

B. With unscaled X_train and X_test 

LogisticRegression 0.9520479254722232,
OverSampling Naive 0.9945453577512777, SMOTE OverSampling 0.9936781215845847, 
ClusterCentroids UnderSampling 0.9936781215845847, 
SMOTEENN Combination - Over and Under Sampling 0.9865149130022852

2. Which model had the best recall score?

Recall Scores-  

For low_risk, all models have a recall score of 0.99
For high_risk, all models except the logistic regression one have 0.99 with LR (LogisitcRegression) Recall score tiny bit lower at 0.98

3. Which model had the best geometric mean score?

Geo Score-  

the geometric mean score is the same (0.99) for all the models (for both low_risk and high_risk outcomes)

Note (for my future ref) Geo "indicates the central tendency or typical value of a set of numbers by using the product of their values"